# Russian Drama Sentiment Analysis
## [RuBert](https://huggingface.co/blanchefort/rubert-base-cased-sentiment)
## [XML Roberta Large](https://huggingface.co/sismetanin/xlm_roberta_large-ru-sentiment-rusentiment?library=transformers)

In [ ]:
import pandas as pd
import torch

In [ ]:
!huggingface-cli login
!pip install datasets

In [ ]:
import pandas as pd

# Login using e.g. `huggingface-cli login` to access this dataset
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("xnsmv/rusdracor_speech")

ValueError: Invalid pattern: '**' can only be an entire path component

In [ ]:
df

,title,author,act,scene,speaker,text,yearNormalized,normalizedGenre,numOfActs,numOfSpeakersMale,numOfSpeakersFemale
0,Лукавин,Писарев,-7,None,Ветрон,"Эй, братцы, выпьем здоровье наших милых, Солид...",1823,Comedy,5,10,2
1,Лукавин,Писарев,-7,None,Гости,Конечно.,1823,Comedy,5,10,2
2,Лукавин,Писарев,-7,None,Ветрон,Подавай.,1823,Comedy,5,10,2
3,Лукавин,Писарев,-7,None,Змейкин,Прекрасное вино!,1823,Comedy,5,10,2
4,Лукавин,Писарев,-7,None,Усладов,Мечи: в другой уж раз напьешься.,1823,Comedy,5,10,2
...,...,...,...,...,...,...,...,...,...,...,...
117037,Фемист и Иеронима,Майков,3,ЯВЛЕНИЕ 7,Фемист,Сокрыл ли славную ты кровь Палеологов?,1775,Tragedy,5,5,1
117038,Фемист и Иеронима,Майков,3,ЯВЛЕНИЕ 7,Клит,"Едва я, государь, исшел из сих чертогов. Внемл...",1775,Tragedy,5,5,1
117039,Фемист и Иеронима,Майков,3,ЯВЛЕНИЕ 7,Фемист,О боже!,1775,Tragedy,5,5,1
117040,Фемист и Иеронима,Майков,3,ЯВЛЕНИЕ 7,Клит,Государь!..,1775,Tragedy,5,5,1


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("sismetanin/xlm_roberta_large-ru-sentiment-rusentiment") # or rubert
model = AutoModelForSequenceClassification.from_pretrained("sismetanin/xlm_roberta_large-ru-sentiment-rusentiment")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

##

In [ ]:
from transformers import pipeline

sentiment_pipeline = pipeline(
    #"sentiment-analysis",
    "text-classification",
    model=model,
    tokenizer=tokenizer,
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    batch_size=16,
    max_length=512,
    truncation=True,
    padding=True
)

Device set to use cpu


In [ ]:
sentiment_pipeline("ОК")

[{'label': 'LABEL_1', 'score': 0.9447472095489502}]

In [ ]:
data = ds["train"]

In [ ]:
texts = data["text"]
results = sentiment_pipeline(texts)

In [ ]:
results = []
for i in range(10):
    text = data[i]["text"]
    result = sentiment_pipeline(text)
    results.append({
        "text": text,
        "sentiment": result[0]["label"],
        "score": result[0]["score"]
    })

print(results)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


[{'text': 'Эй, братцы, выпьем здоровье наших милых, Солидных, ветреных, веселых и унылых: Они все - женщины, все милы нам равно.', 'sentiment': 'NEUTRAL', 'score': 0.6120244860649109}, {'text': 'Конечно.', 'sentiment': 'NEUTRAL', 'score': 0.816451907157898}, {'text': 'Подавай.', 'sentiment': 'NEUTRAL', 'score': 0.8105477690696716}, {'text': 'Прекрасное вино!', 'sentiment': 'NEGATIVE', 'score': 0.7514878511428833}, {'text': 'Мечи: в другой уж раз напьешься.', 'sentiment': 'NEGATIVE', 'score': 0.7515494227409363}, {'text': 'Погодите. Вы с вашею игрой и выпить не дадите. Ведь эдак надоест игра.', 'sentiment': 'NEGATIVE', 'score': 0.751232922077179}, {'text': 'Какой пустяк! Не меньше моего ты карты любишь.', 'sentiment': 'POSITIVE', 'score': 0.9533298015594482}, {'text': 'Так: Я карты сам люблю, но это слишком много.', 'sentiment': 'POSITIVE', 'score': 0.7539337277412415}, {'text': 'Давно ли Змейкин наш судить так начал строго?', 'sentiment': 'NEUTRAL', 'score': 0.8234673738479614}, {'text

In [ ]:
res = pd.DataFrame(results)
res

,label,score
0,LABEL_3,0.707513
1,LABEL_1,0.971675
2,LABEL_1,0.965023
3,LABEL_2,0.997097
4,LABEL_1,0.909799
...,...,...
117037,LABEL_1,0.781606
117038,LABEL_3,0.630807
117039,LABEL_3,0.597723
117040,LABEL_1,0.942564


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#file_path = "/content/drive/senti_res.xlsx"
res.to_excel("senti_res.xlsx", index=False)

In [ ]:
import pandas as pd

# Login using e.g. `huggingface-cli login` to access this dataset
df = pd.read_parquet("hf://datasets/xnsmv/rusdracor_speech/speech_meta.parquet")

In [ ]:
df

,title,author,act,scene,speaker,text,yearNormalized,normalizedGenre,numOfActs,numOfSpeakersMale,numOfSpeakersFemale
0,Лукавин,Писарев,-7,None,Ветрон,"Эй, братцы, выпьем здоровье наших милых, Солид...",1823,Comedy,5,10,2
1,Лукавин,Писарев,-7,None,Гости,Конечно.,1823,Comedy,5,10,2
2,Лукавин,Писарев,-7,None,Ветрон,Подавай.,1823,Comedy,5,10,2
3,Лукавин,Писарев,-7,None,Змейкин,Прекрасное вино!,1823,Comedy,5,10,2
4,Лукавин,Писарев,-7,None,Усладов,Мечи: в другой уж раз напьешься.,1823,Comedy,5,10,2
...,...,...,...,...,...,...,...,...,...,...,...
117037,Фемист и Иеронима,Майков,3,ЯВЛЕНИЕ 7,Фемист,Сокрыл ли славную ты кровь Палеологов?,1775,Tragedy,5,5,1
117038,Фемист и Иеронима,Майков,3,ЯВЛЕНИЕ 7,Клит,"Едва я, государь, исшел из сих чертогов. Внемл...",1775,Tragedy,5,5,1
117039,Фемист и Иеронима,Майков,3,ЯВЛЕНИЕ 7,Фемист,О боже!,1775,Tragedy,5,5,1
117040,Фемист и Иеронима,Майков,3,ЯВЛЕНИЕ 7,Клит,Государь!..,1775,Tragedy,5,5,1


In [ ]:
df.to_excel("clean_data.xlsx", index=False)

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="sismetanin/xlm_roberta_large-ru-sentiment-rusentiment")

In [ ]:
pipe("Мрак")

[{'label': 'LABEL_0', 'score': 0.9865902662277222}]

In [ ]:
pipe("Погодите. Вы с вашею игрой и выпить не дадите. Ведь эдак надоест игра.")

[{'label': 'LABEL_0', 'score': 0.786068856716156}]

In [ ]:
pipe("Плохо")

[{'label': 'LABEL_0', 'score': 0.9938502311706543}]

In [ ]:
pipe("1q2w2e3")

[{'label': 'LABEL_1', 'score': 0.5286687612533569}]

In [ ]:
pipe("Привет")

[{'label': 'LABEL_4', 'score': 0.9945758581161499}]

In [ ]:
pipe("1")

[{'label': 'LABEL_1', 'score': 0.982679009437561}]

In [ ]:
pipe("Безумный И злой старик! На свете все живое Должно любить. Снегурочку в неволе Не даст тебе томить родная мать")

[{'label': 'LABEL_3', 'score': 0.49104827642440796}]